In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
%matplotlib inline

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
# Epoch과 Batch_size 선언
EPOCHS = 100
BATCH_SIZE = 16


In [4]:
df=pd.read_excel('../optimal_data3_1/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
Count=int(322/10)*83
pred_list=[]

In [5]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.input_layer = nn.Linear(6, 128)
        self.hidden_layer1 = nn.Linear(128, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.output_layer   = nn.Linear(128,3)

        
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        out =  F.relu(self.input_layer(x))
        out =  F.relu(self.hidden_layer1(out))
        out= self.dropout(out)
        out =  F.relu(self.hidden_layer2(out))
        out= self.dropout(out)
        out =  self.output_layer(out)
        return out 

In [6]:
# model=DNN().to(DEVICE)
# # 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
# optimizer    = optim.Adam(model.parameters(), lr=0.01)

# # 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
# loss_fn = nn.CrossEntropyLoss()

### 학습하기

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target=data.to(DEVICE), target.to(DEVICE)
        #매 반복(iteration) 마다 기울기를 계산하기 위해 zero_grad() 호출
        target=target.long()
        optimizer.zero_grad()
        # 실제 모델의 예측값(output) 받아오기
        output=model(data)
        #정답 데이터와의 CrossEntropyLoss 계산
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = F.cross_entropy(output,torch.max(target,1)[1].long())
        
        #기울기 계산
        loss.backward()
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()


### 테스트 하기

In [8]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()
    # 필요한 변수 초기화
    # Test과정에서의 Loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0
    pred_list=[]
    pred_array=[]
    with torch.no_grad(): # 평가 과정에서는 기울기를 계산하지 않으므로, no_grad명시
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            _, pred = output.max(dim=1)
            pred_array=pred.tolist()
            pred_list+=pred_array
                                    # confusion matrix를 위해 pred 리턴 값
           
            # 모든 오차 더하기
            test_loss +=  F.cross_entropy(output, torch.max(target,1)[1], reduction="sum").item()
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측 클래스(pred)과 정답 클래스를 비교하여 일치할 경우 correct에 1을 더합니다.
            target=torch.argmax(target, dim=1)
            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    #정확도 계산
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy, pred_list

In [9]:
#한 사람당 데이터 수
Count_1=int(181*0.1)
#한 사람당 데이터 수
Count_2=181

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]



In [ ]:
for i in range(10):
    model=DNN().to(DEVICE)
    # 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
    optimizer    = optim.Adam(model.parameters(), lr=0.01)

    # 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
    loss_fn = nn.CrossEntropyLoss()
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    y_test_list=[]
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
        
    
    print('SMOTE 적용 전 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 전 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
    print('SMOTE 적용 후 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 후 Test 레이블 값 분포: \n', y_test.value_counts())    
    
    
    #모든 데이터 torch로 변환
    X_train = torch.FloatTensor(X_train.to_numpy())
    X_test = torch.FloatTensor(X_test.to_numpy())    
    y_train=y_train.to_numpy()
    y_train=np.ravel(y_train, order='C')
    y_train = torch.LongTensor(y_train)
    y_test=y_test.to_numpy()
    y_test=np.ravel(y_test, order='C')
    y_test = torch.LongTensor(y_test)
    
    # 원핫 인코딩
    y_train=F.one_hot(y_train).long()
    y_test=F.one_hot(y_test).long()
    

    # train_dataset, test_dataset을 구별하여 정의
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset=TensorDataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False)
    

    for epoch in range(1, EPOCHS + 1):
        train(model, train_dataloader, optimizer)
    
    test_loss, test_accuracy, predict = evaluate(model, test_dataloader)
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))
        
#     print("[{}]Predict : {}".format(i,predict))
    #Accuracy
    #test label 데이터 토치에서 list로 변경
    y_test_list=torch.argmax(y_test,dim=1).tolist()

    accuracy=accuracy_score(y_test_list, predict) * 100
    print("[{}]Accuracy : {}".format(i,accuracy))   
    #f1score
    f1 = f1_score(y_test_list,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    p_rlist=sk(y_test_list,predict,average='weighted')
    print("[{}]Precision : {}".format(i,p_rlist[0]))
    print("[{}]Recall : {}".format(i,p_rlist[1]))
    print()
     #결과 배열에 넣기
    Result[i][0]=accuracy
    Result[i][1]=f1
    Result[i][2]=p_rlist[0]
    Result[i][3]=p_rlist[1]
    del accuracy
    del f1
    del p_rlist

SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    49915
1.0     1621
2.0      950
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    5199
1.0     368
2.0     229
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (15597, 6) (15597, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    49915
1.0    49915
2.0    49915
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    5199
1.0    5199
2.0    5199
dtype: int64
[100] Test Loss: 4.8047, Accuracy: 33.33%
[0]Accuracy : 33.33333333333333
[0]F1score : 0.16666666666666666
[0]Precision : 0.1111111111111111
[0]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    49915
1.0     1621
2.0      950
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    5157
1.0     383
2.0     256
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (15471, 6) (15471, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    49915
1.0    49915
2.0    49915
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    5157
1.0    5157
2.0    5157
dtype: int64
[100] Test Loss: 4.8047, Accuracy: 33.33%
[1]Accuracy : 33.33333333333333
[1]F1score : 0.16666666666666666
[1]Precision : 0.1111111111111111
[1]Recall : 0.3333333333333333



C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    49915
1.0     1621
2.0      950
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    5349
1.0     287
2.0     160
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (16047, 6) (16047, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    49915
1.0    49915
2.0    49915
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    5349
1.0    5349
2.0    5349
dtype: int64


In [ ]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

In [ ]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix2.xlsx')

In [ ]:
y_train

In [ ]:
y_test